In [79]:
'''Plan:
1. Load libraries, load data.
2. Preliminary EDA, dealing with missing values, merging train and test.
3. EDA, deleting variables.
4. Feature engineering, ohc.
5. Sample formation.
6. Feature scaling.
7. Model fitting.
8. Performance evaluation.
9. Predictions.
'''

# aside:
# when coding for interview ML purposes or Kaggle, never drop any obervations!
# you will have to make predictions for all obs in test sample.

# correct way to deal with missing obs and merge train and test samples:
# 1. Load both samples.
# 2. Impute missing values in both samples, using train sample to impute missing values.
# 3. Concatentate them into df.


### 1. Load libraries ###

import numpy as np
import pandas as pd
import os, warnings, random, time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer

from xgboost import XGBClassifier

warnings.filterwarnings("ignore")
#os.getcwd()

In [87]:
### Load data ###

train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
print(train.shape)

test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

train = train.sample(n=50000)

(957919, 120)


In [88]:
### 2. ###

train.shape
train.describe()

# are there numerical features?

un_colval = pd.DataFrame([[x,len(train[x].unique())] for x in train.columns], columns = ['colname', 'n_unique'])
un_colval.loc[un_colval.n_unique < 100]
# all columns contain numerical features

,colname,n_unique
119,claim,2


In [89]:
#train.describe()
#test.describe()

# imputing missing values #

colmnames = train.columns
test['claim'] = np.nan
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(train)
train = pd.DataFrame(imp.transform(train))
test = pd.DataFrame(imp.transform(test))
#train.describe()

In [90]:
train['sample'] = 'train'
test['sample'] = 'pred'
df = pd.concat([train, test])
#df.colnames = list(colmnames)+'sample'
df

,0,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,sample
0,153815.0,0.101330,0.47947,15385.00,0.246190,0.27297,-0.869000,2042.90,41499.0,3.587700e+15,...,1.5528,-1.75110,2.58050,792.06,1.1450,2.949200e+15,181.13,1.819100,0.0,train
1,664221.0,0.093230,0.26313,3511.10,0.015731,0.47427,-1.424600,1129.00,192410.0,-7.741900e+12,...,4.2677,-1.25610,13.16800,105410.00,1.0892,2.573600e+13,231.59,0.178190,0.0,train
2,666309.0,0.018645,0.32663,886.63,0.013459,0.24345,0.246830,295.61,467920.0,1.999900e+15,...,1.1023,18.14900,-8.60080,172500.00,1.1257,1.748600e+16,248.96,0.084167,1.0,train
3,896941.0,0.083500,0.42187,728.63,0.188360,0.36629,-2.261100,341.64,187180.0,8.379500e+12,...,1.6407,-1.82440,0.89801,238650.00,1.0732,4.401500e+15,10525.00,0.590930,0.0,train
4,33685.0,0.079317,0.29726,-2453.00,0.027737,0.22532,-0.619640,569.76,87058.0,1.317400e+13,...,1.5598,1.35600,0.79779,10758.00,1.2744,3.976300e+16,1445.60,0.474480,0.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493469,1451388.0,-0.009112,0.30819,637.64,0.778200,0.41415,-1.068500,651.22,985000.0,6.079700e+15,...,2.3325,0.11226,-5.92380,19284.00,1.1559,8.163900e+16,857.09,1.563300,0.0,pred
493470,1451389.0,0.088922,0.48265,6924.10,0.025963,0.35540,-0.870200,2514.20,18004.0,6.073500e+14,...,1.7005,97.81300,4.37930,-2432.00,1.0707,4.691800e+16,7497.10,0.670750,0.0,pred
493471,1451390.0,0.140620,0.48475,1797.10,0.147020,0.28803,-1.407100,434.03,333050.0,2.351000e+15,...,1.6827,1.72910,8.06330,2471.40,1.1725,7.900900e+16,2904.60,0.180050,0.0,pred
493472,1451391.0,0.168000,0.35176,454.79,0.164580,0.16983,0.323850,2331.20,223980.0,-2.795300e+12,...,1.3531,155.21000,13.96300,-11.44,1.1946,-1.770600e+14,6763.10,0.332230,0.0,pred


In [96]:
newcolnames = list(colmnames) + ['sample']
df.columns = newcolnames
df

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f111,f112,f113,f114,f115,f116,f117,f118,claim,sample
0,153815.0,0.101330,0.47947,15385.00,0.246190,0.27297,-0.869000,2042.90,41499.0,3.587700e+15,...,1.5528,-1.75110,2.58050,792.06,1.1450,2.949200e+15,181.13,1.819100,0.0,train
1,664221.0,0.093230,0.26313,3511.10,0.015731,0.47427,-1.424600,1129.00,192410.0,-7.741900e+12,...,4.2677,-1.25610,13.16800,105410.00,1.0892,2.573600e+13,231.59,0.178190,0.0,train
2,666309.0,0.018645,0.32663,886.63,0.013459,0.24345,0.246830,295.61,467920.0,1.999900e+15,...,1.1023,18.14900,-8.60080,172500.00,1.1257,1.748600e+16,248.96,0.084167,1.0,train
3,896941.0,0.083500,0.42187,728.63,0.188360,0.36629,-2.261100,341.64,187180.0,8.379500e+12,...,1.6407,-1.82440,0.89801,238650.00,1.0732,4.401500e+15,10525.00,0.590930,0.0,train
4,33685.0,0.079317,0.29726,-2453.00,0.027737,0.22532,-0.619640,569.76,87058.0,1.317400e+13,...,1.5598,1.35600,0.79779,10758.00,1.2744,3.976300e+16,1445.60,0.474480,0.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493469,1451388.0,-0.009112,0.30819,637.64,0.778200,0.41415,-1.068500,651.22,985000.0,6.079700e+15,...,2.3325,0.11226,-5.92380,19284.00,1.1559,8.163900e+16,857.09,1.563300,0.0,pred
493470,1451389.0,0.088922,0.48265,6924.10,0.025963,0.35540,-0.870200,2514.20,18004.0,6.073500e+14,...,1.7005,97.81300,4.37930,-2432.00,1.0707,4.691800e+16,7497.10,0.670750,0.0,pred
493471,1451390.0,0.140620,0.48475,1797.10,0.147020,0.28803,-1.407100,434.03,333050.0,2.351000e+15,...,1.6827,1.72910,8.06330,2471.40,1.1725,7.900900e+16,2904.60,0.180050,0.0,pred
493472,1451391.0,0.168000,0.35176,454.79,0.164580,0.16983,0.323850,2331.20,223980.0,-2.795300e+12,...,1.3531,155.21000,13.96300,-11.44,1.1946,-1.770600e+14,6763.10,0.332230,0.0,pred


In [98]:
### 3. ###

# we have many variables, are all of them useful? lets see correlation

#cors = df.corr()
#cors.loc[((cors > 0.8)&(cors<1)).any(1)]
# there are no pairwise correlations above 80%

In [99]:
### 4. ###

# check skew and possibly transform some variables #


In [101]:
### 5. ###

y_train = df.loc[df['sample']=='train',['claim']]
X_train = df.drop(columns=['claim'])
X_train = X_train.loc[X_train['sample']=='train']
X_pred = X_train.loc[X_train['sample']=='test']

print(y_train.shape, X_train.shape)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=2)

print(y_train.shape, X_train.shape, X_test.shape)

(50000, 1) (50000, 120)
(45000, 1) (45000, 120) (5000, 120)


In [106]:
X_train.drop(columns = ['sample'], inplace=True)
X_test.drop(columns = ['sample'], inplace=True)
X_pred.drop(columns = ['sample'], inplace=True)

In [110]:
X_train.shape

(45000, 119)

In [108]:
### 6. Scaling ###

ss = StandardScaler()

for i in X_train.columns:
    X_train[[i]] = ss.fit_transform(X_train[[i]])
    X_test[[i]] = ss.transform(X_test[[i]])

X_train.describe()  

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
count,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,...,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04
mean,-6.315935e-19,-9.158106e-18,-4.736952e-19,-4.736952e-19,2.210577e-18,7.263326e-18,-6.631732e-18,2.242157e-17,8.210716e-18,-2.147418e-17,...,3.434290e-17,-2.100049e-17,2.431635e-17,1.326346e-17,1.373716e-17,3.394815e-17,-6.000139e-18,1.863201e-17,-2.257947e-17,1.263187e-17
std,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,...,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00
min,-1.726834e+00,-5.164668e+00,-2.532113e+00,-2.053232e+00,-1.264671e+00,-2.161037e+00,-5.658343e+00,-1.450866e+00,-1.172658e+00,-1.269315e+00,...,-1.458633e+00,-4.547718e+00,-1.983510e+00,-1.006662e+00,-2.827017e+00,-1.260819e+00,-2.679426e+00,-7.276203e-01,-1.363507e+00,-1.649189e+00
25%,-8.645809e-01,-4.566085e-01,-4.165152e-01,-5.616219e-01,-7.746882e-01,-4.277016e-01,-4.907341e-01,-8.891264e-01,-8.255877e-01,-7.649079e-01,...,-9.379118e-01,-4.770088e-01,-6.432873e-01,-5.283349e-01,-6.089870e-01,-6.502929e-01,-5.327122e-01,-6.288666e-01,-8.351701e-01,-6.854593e-01
50%,-4.789416e-03,-4.955262e-03,2.927658e-01,-4.292499e-01,-2.974517e-01,1.549689e-01,-1.429745e-01,-1.357785e-01,-2.480328e-01,-5.552258e-01,...,-1.177348e-01,2.840643e-01,-4.535448e-01,-4.772909e-01,-7.867918e-02,-4.700340e-01,-2.697192e-01,-4.329457e-01,-2.305307e-01,-2.102455e-01
75%,8.654717e-01,5.843884e-01,7.607338e-01,4.185011e-02,4.383337e-01,7.338615e-01,4.682288e-01,6.785341e-01,5.250771e-01,5.519983e-01,...,7.410163e-01,7.807398e-01,4.434169e-01,-1.152250e-01,5.009280e-01,2.557402e-01,2.686640e-01,1.418491e-01,6.805811e-01,4.479534e-01
max,1.732506e+00,6.470226e+00,1.175642e+00,5.330060e+00,5.119528e+00,1.672848e+00,4.970657e+00,2.965770e+00,4.338003e+00,3.657971e+00,...,2.397132e+00,1.145887e+00,2.757637e+00,4.319451e+00,4.531148e+00,4.890537e+00,5.382418e+00,4.091651e+00,2.884070e+00,5.151605e+00


In [109]:
### 7. Model fitting ###

time1 = time.time()

log_rg = LogisticRegression()

grid_values = {'penalty': ['l2'], 'C': [0.00001, 0.0001, 0.001, 0.01]}

lr = GridSearchCV(log_rg, param_grid = grid_values, cv=4)
lr.fit(X_train, y_train)

print('logistic', lr.best_score_, lr.best_params_, time.time()-time1)
# after dropping missing I had 86%...

logistic 0.5230222222222223 {'C': 0.0001, 'penalty': 'l2'} 2.329680919647217


In [ ]:
time1 = time.time()

svm = svm.SVC(kernel='rbf')

grid_values = {'C':[0.01, 0.1, 1, 5, 10]}

svm = GridSearchCV(svm, param_grid = grid_values, cv=2)
svm.fit(X_train, y_train)

print('SVM', svm.best_score_, svm.best_params_, time.time()-time1)

In [33]:
time1 = time.time()

xgbcl = XGBClassifier()

grid_values = {'eta':[0.01, 0.1], 'max_depth':[2,3,5]}

xgb = GridSearchCV(xgbcl, param_grid = grid_values, cv=2)
xgb.fit(X_train, y_train)

print('XGBoost', xgb.best_score_, xgb.best_params_, time.time()-time1)

[18:26:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:26:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:26:44] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:27:01] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

KeyboardInterrupt: 